# Method A fit quick test

This notebook wires up the local Python fitting drivers and runs against `data/work.csv` (the 2D histogram exported as long-form CSV). It now supports a quick sanity mode (single model evaluation) and pre/post 2D histogram plotting.

In [ ]:
from pathlib import Path
import sys

# Resolve repository root whether the notebook is launched from repo root or python_code/
cwd = Path.cwd().resolve()
repo_root = cwd if (cwd / 'python_code').exists() else cwd.parent
assert (repo_root / 'python_code').exists(), f'Could not find repo root from {cwd}'

python_code_dir = repo_root / 'python_code'
data_csv = repo_root / 'data' / 'work.csv'

if str(python_code_dir) not in sys.path:
    sys.path.insert(0, str(python_code_dir))

print('repo_root   :', repo_root)
print('python_code :', python_code_dir)
print('data_csv    :', data_csv)
print('csv exists? :', data_csv.exists())


In [ ]:
# Dependency check: this notebook expects local Python scientific deps to be available
missing = []
for pkg in ('numpy', 'iminuit', 'matplotlib'):
    try:
        __import__(pkg)
    except Exception:
        missing.append(pkg)

if missing:
    raise RuntimeError(
        'Missing Python dependencies: ' + ', '.join(missing) + '\n'
        'Install them in your notebook kernel environment before running the fit.'
    )

from fit_driver import fit_noE

print('Dependencies loaded successfully.')


In [ ]:
# Optional quick preview of the long-form teardrop CSV using stdlib only
import csv

with open(data_csv, newline='') as f:
    reader = csv.DictReader(f)
    rows = [next(reader) for _ in range(5)]

rows


In [ ]:
# Run options
quick_sanity = True   # True = one fast model evaluation (no minimization)
plot_histograms = True

methodA, best, minuit = fit_noE(
    [str(data_csv)],
    verbose=False,
    quick_sanity=quick_sanity,
    plot_histograms=plot_histograms,
)

param_names = [
    'a_ev','b_Fierz','log10N','costhetamin','LNabM5','alpha','beta','gamma','eta',
    'z0_center','z0_width','missdet','tailfrac','tailVal',
    'hvMapMin1','hvMap0','hvMap1','hvMap2','hvMap3','hvMap4',
    'calEe','EeNonLinearity','sigmaEe_keV'
]

result = {name: float(val) for name, val in zip(param_names, best)}
if minuit is None:
    print('Mode: quick_sanity (single evaluation, no migrad minimization).')
else:
    print('Fit valid :', minuit.fmin.is_valid)
    print('chi2      :', float(minuit.fval))
    print('ndof      :', int(minuit.ndof) if hasattr(minuit, 'ndof') else 'n/a')
result
